In [1]:
#数据读取与预处理
#本程序使用Kim2014的TextCNN对Malo2014的FinancialPhraseBank数据集进行分类
with open('/root/study/Data/FinancialPhraseBank/Sentences_75Agree.txt','r')as f:
    FPB_75_in = f.read() #Malo的数据特色，分为不同置信度的样本
with open('/root/study/Data/FinancialPhraseBank/Sentences_AllAgree.txt','r')as f:
    FPB_100_in = f.read()

In [2]:
FPB_75_in[:200]

'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral\nWith the new production plant the company would increase its cap'

In [3]:
FPB_75_lines = FPB_75_in.split('\n')
FPB_75_lines[0].split('@')

['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'neutral']

In [4]:
from string import punctuation

FPB_75_data = []
FPB_75_label = []
tags = {'positive':2, 'neutral':1, 'negative':0} #标签映射
for line in FPB_75_lines[:-1]:
    line = line.lower() #小写化
    data_line = line.split('@')[0] #数据部分
    FPB_75_label.append(tags[line.split('@')[1]]) #标签部分
    data_line = ''.join([c for c in data_line if c not in punctuation])#去除标点符号
    data_l = data_line.split(' ') #分词
    data_l = [c for c in data_l if len(c)>0] #Malo这份数据的特别之处，标点前有空格，去除空格
    FPB_75_data.append(data_l) 

In [5]:
print(FPB_75_data[0],'\n',FPB_75_label[0]) #查看数据和标签

['according', 'to', 'gran', 'the', 'company', 'has', 'no', 'plans', 'to', 'move', 'all', 'production', 'to', 'russia', 'although', 'that', 'is', 'where', 'the', 'company', 'is', 'growing'] 
 1


In [6]:
FPB_100_lines = FPB_100_in.split('\n')
FPB_100_data = []
FPB_100_label = []
tags = {'positive':2, 'neutral':1, 'negative':0} #标签映射
for line in FPB_100_lines[:-1]:
    line = line.lower() #小写化
    data_line = line.split('@')[0] #数据部分
    FPB_100_label.append(tags[line.split('@')[1]]) #标签部分
    data_line = ''.join([c for c in data_line if c not in punctuation])#去除标点符号
    data_l = data_line.split(' ') #分词
    data_l = [c for c in data_l if len(c)>0] #Malo这份数据的特别之处，标点前有空格，去除空格
    FPB_100_data.append(data_l) 

In [7]:
FPB_data_raw = FPB_75_data + FPB_100_data
FPB_label_raw = FPB_75_label + FPB_100_label

In [8]:
FP_len = []
for li in FPB_data_raw:
    FP_len.append(len(li))

import pandas as pd

pd.DataFrame(FP_len).describe() #查看数据长度分布，确定切断尺寸

,0
count,5717.000000
mean,20.027287
std,8.876611
min,1.000000
25%,13.000000
50%,19.000000
75%,26.000000
max,52.000000


In [9]:
#由于是拼接的100一致和75一致，100在后面，避免这个影响要先random
import random
random.seed(42)
raw_idx = list(range(len(FPB_data_raw)))
random.shuffle(raw_idx)

FPB_data = []
FPB_label = []
for idx in raw_idx:
    FPB_data.append(FPB_data_raw[idx])
    FPB_label.append(FPB_label_raw[idx])


In [10]:
FPB_label_raw[0]

1

加载预训练w2v模型

In [11]:
from gensim.models import KeyedVectors
embed_lookup = KeyedVectors.load_word2vec_format('/root/study/Data/model/GoogleNews-vectors-negative300-SLIM.bin',binary=True) #加载预训练w2v模型
embed_lookup.key_to_index['hello'] #测试一下是否加载成功

18680

In [12]:
def get_vec_index(wordl):#本函数获得所有词的w2v的index
    resl = []
    for wi in wordl:
        try:
            idx = embed_lookup.key_to_index[wi] #获取词向量索引
        except Exception:
            idx = 0
        resl.append(idx)
    return resl

In [13]:
FPB_vec_index = list(map(get_vec_index,FPB_data))

In [14]:
import numpy as np
def padding_tokens(raw_vec,padding_len=60,sq_len=None):#对所有数据进行补齐，在本例中所有数据长度为60
    tlen = len(raw_vec)
    if sq_len is not None:
        sq_len.append(tlen)
    if tlen>=padding_len:
        return raw_vec[:padding_len]
    
    for li in range(padding_len-tlen):
        raw_vec.append(0)
    return np.array(raw_vec)

In [15]:
sq_len = []
FPB_vec_index_padding = list(map(lambda x: padding_tokens(x,padding_len=60,sq_len=sq_len),FPB_vec_index))

In [16]:
features = np.array(FPB_vec_index_padding)
labels = np.array(FPB_label)
features[:10,:6] #查看前10条数据的向量表示，60维度的词向量

array([[   796,    864,    563,      0,  17001,      0],
       [   468,      4,    584, 178363,      8,      0],
       [     0,   6638,      0,     10,      9,     54],
       [     9,   1179,    130,    840,      0,    682],
       [182879,    730,      0,      0,   1239,   6407],
       [     9,     94,   1006,      3,      0,      0],
       [     9,    429,      3,    249,      0,     14],
       [     0,      1,      9,     32,      0,   1470],
       [     0,      0,      0,      0,    864,     93],
       [   172,    796,    864,      0,   5119,      1]])

In [17]:
#标准化数据
#切分训练集、测试集、验证集
train_frac = 0.8
valid_frac = 0.9

train_idx = int(len(features)*train_frac)
valid_idx = int(len(features)*valid_frac)

train_x,valid_x,test_x = features[:train_idx],features[train_idx:valid_idx],features[valid_idx:]
train_y,valid_y,test_y = labels[:train_idx],labels[train_idx:valid_idx],labels[valid_idx:]
train_sq_len,valid_sq_len,test_sq_len = sq_len[:train_idx],sq_len[train_idx:valid_idx],sq_len[valid_idx:]


In [18]:
import torch
from torch.utils.data import TensorDataset, DataLoader
#制作标准Tensor数据集、包含特征、标签
train_dataset = TensorDataset(torch.tensor(train_x, dtype=torch.long), torch.tensor(train_y, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(valid_x, dtype=torch.long), torch.tensor(valid_y, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(test_x, dtype=torch.long), torch.tensor(test_y, dtype=torch.long))
#tensordataset可以把数据转化为Tensor形式，便于后续处理
batch_size = 50

#随机化与Batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [19]:
#保存数据，以供后期使用
torch.save({"data":train_dataset.tensors[0], "labels":train_dataset.tensors[1],"lengths":train_sq_len}, '/root/study/Data/Malo_train.pth')
torch.save({"data":valid_dataset.tensors[0], "labels":valid_dataset.tensors[1],"lengths":valid_sq_len}, '/root/study/Data/Malo_valid.pth')
torch.save({"data":test_dataset.tensors[0], "labels":test_dataset.tensors[1],"lengths":test_sq_len}, '/root/study/Data/Malo_test.pth')


In [20]:
len(valid_x) #查看验证集的长度，应该是0.1*数据集长度

572

In [21]:
#检查dataloader的输出
for batch_idx,(x,y) in enumerate(train_loader):
    print(f"训练集-Batch {batch_idx}:")
    print(f"Features: {x.shape}, Labels: {y.shape}")
    break #查看第一个批次的结果即可
#x的shape是(batch_size, seq_len)，y的shape是(batch_size,)，即每个batch的特征和标签

训练集-Batch 0:
Features: torch.Size([50, 60]), Labels: torch.Size([50])


In [22]:
#数据预处理完成，接下来可以进行模型构建和训练
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, embed_model, vocab_size, output_size,embed_dim,
                 num_filters=100, kernel_sizes=[3,4,5], embed_freeze=True,dropout=0.5):
        super(TextCNN, self).__init__()
        #设定参数
        self.num_filters = num_filters #卷积核的数量
        self.embed_dim = embed_dim #词向量会被映射为多少维

        #  1.词嵌入层
        self.embed = nn.Embedding(vocab_size, embed_dim)
        # 加载预训练的权重
        self.embed.weight = nn.Parameter(torch.from_numpy(embed_model.vectors))
        # 冻结嵌入层的训练，设定为false则一起训练
        if embed_freeze:
            self.embed.weight.requires_grad = False
        
        #  2.卷积层
        self.convs_1d = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embed_dim),padding = (k-2, 0)) for k in kernel_sizes
        ])#并联多个二维卷积层，每个卷积核的宽度设定为embed_dim，高度为不同的k值
        #  3.全连接层
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, output_size)
        #  4. dropout层
        self.dropout = nn.Dropout(dropout)

    def conv_and_pool(self,x,conv): #定义一个卷积和池化的函数
        x = F.relu(conv(x)).squeeze(3) #卷积后激活,对于每个conv(一维卷积)，卷积后的结果第四个维度会是1，可以删掉squeeze）
        #四个维度分别是batch_size,num_filters,sequence_length,1
        x_max = F.max_pool1d(x, x.size(2)).squeeze(2) #最大池化，池化后结果的维度是(batch_size,num_filters)，即每个卷积核的最大值
        #然后会删掉第三个维度
        return x_max
        #返回的是(batch_size, num_filters)，即每个卷积核的最大值

    def forward(self, x):

        # 1.词嵌入
        embeds = self.embed(x) #输入x x是一个batch的索引，维度是(batch_size, seq_len),seq_len是每个句子的长度
        #输出embeds的维度是(batch_size, seq_len, embed_dim),嵌入向量每个元素的维度是embed_dim
        embeds = embeds.unsqueeze(1) #增加一个维度，变为(batch_size, 1, seq_len, embed_dim)，1是单通道
        #目的是为了适应卷积层的输入格式，卷积层需要四维输入(batch_size, channels, height, width)

        # 2.卷积和池化
        conv_results = [self.conv_and_pool(embeds, conv) for conv in self.convs_1d]
        #self.convs_ld是一个包含多个2d卷积核的moudulelist，每个卷积核负责不同大小的窗口kernel_size
        #对每个卷积核进行卷积和池化(通过self.conv_and_pool提取特征)，
        #得到每个卷积核生成的形状为(batch_size, num_filters)的特征向量
        #结果是一个列表，每个元素是(batch_size, num_filters)，即每个卷积核的最大值

        # 3.拼接卷积结果
        x = torch.cat(conv_results,1) #将所有卷积核的结果拼接在一起，变为(batch_size, num_filters * len(kernel_sizes))
        #即每个卷积核的最大值拼接在一起

        # 4. dropout
        x = self.dropout(x) #对拼接后的结果进行dropout，防止过拟合

        # 5.全连接层
        x = self.fc(x) #将拼接后的结果输入全连接层，输出的维度是(batch_size, output_size)
        #output_size是分类的数量
        return F.log_softmax(x, dim=1) #使用log_softmax进行分类，dim=1表示对每一行进行softmax   


In [23]:
#训练函数与测试函数仍然可以复用
#定义一个训练函数
def train(args, model, device, train_loader, optimizer, epoch):
    # args：一般是一个包含超参数的对象或命名空间（如 batch_size, learning_rate 等），用于控制训练过程。
    # model：传入的神经网络模型，需要被训练。
    # device：设备信息（如 cpu 或 cuda），决定模型和数据在哪个硬件上运行。
    # train_loader：数据加载器，包含训练数据的批次，用于迭代访问每个 batch。
    # optimizer：优化器，用于更新模型的参数（如 SGD、Adam）。
    # epoch：当前的训练轮次编号，用于记录进度。
    
    model.train()  #将模型切换为训练模式。pytorch中有train和eval两个模式，一些机制如dropout Batchnorm仅在train模式中生效
    for batch_idx, (data, target) in enumerate(train_loader):  #从train_loader中加载数据，data、target分别为数据和标签，batch_idx为批次索引从0开始
        data, target = data.to(device), target.to(device)  #将数据和标签转移到指定的设备（CPU 或 GPU）。如果数据和标签不在相同设备上则会报错
        optimizer.zero_grad()     #清空优化器中所有参数的梯度缓存。PyTorch 的默认行为是 梯度累积（每次调用 backward() 会将新梯度累加到现有的梯度上），因此需要在每次迭代开始前清空梯度。
        output = model(data)      #使用模型对当前批次的输入 data 进行前向传播，得到预测结果 output
        loss = F.nll_loss(output, target)  #计算预测值与真实标签之间的误差（损失）：F.nll_loss是 PyTorch 提供的负对数似然损失函数（Negative Log Likelihood Loss）
        loss.backward()           #loss反向传播
        optimizer.step()          #根据计算得到的梯度，利用优化器更新模型的参数

#定义一个测试函数
def test(model, device, test_loader):
    # model：需要评估的神经网络模型。
    # device：用于运行模型和数据的设备（如 CPU 或 GPU）。
    # test_loader：数据加载器，提供测试数据集，通常不会打乱顺序。
    
    model.eval()  #将模型切换为eval模式
    test_loss = 0
    correct = 0
    with torch.no_grad():  #禁用自动求导功能，节省资源，加速运行
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)    #将数据和标签转移到指定的设备（CPU 或 GPU）。如果数据和标签不在相同设备上则会报错
            output = model(data)    #将数据输入模型，得到预测结果 output
            test_loss += F.nll_loss(output, target, reduction='sum').item()  #使用负对数似然损失（nll_loss）计算当前批次的损失，并将其加总
            pred = output.argmax(dim=1, keepdim=True)   #从模型输出中选择每个样本中最大值对应的索引作为预测类别。
                                                        #argmax(dim=1)：在每行（样本）的预测值中找到概率最大的类别索引。
                                                        #keepdim=True：保持结果的维度一致，方便与标签进行比较。
            correct += pred.eq(target.view_as(pred)).sum().item()  #累计正确预测数量；pred.eq(target.view_as(pred))：比较预测值 pred 和真实值 target 是否相等，返回一个布尔张量。

    test_loss /= len(test_loader.dataset)  #总测试损失除以测试集样本总数，得到平均损失。

    print("=" * 50)
    print("Evaluation Results:")
    print(f" - Average Loss: {test_loss:.4f}")
    print(f" - Accuracy: {correct}/{len(test_loader.dataset)} "
          f"({100. * correct / len(test_loader.dataset):.2f}%)")
    print("=" * 50)

In [24]:
epochs = 6  #定义最大训练周期
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #定义设备,CPU或者GPU（Cuda）
print(device)

cuda


In [25]:
pretrained_words = []
for word in embed_lookup.key_to_index:
    pretrained_words.append(word)

In [26]:
vocab_size = len(pretrained_words)
output_size = 3 
embed_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors
num_filters = 100
kernel_sizes = [3, 4, 5]

model = TextCNN(embed_lookup, vocab_size, output_size, embed_dim,
                   num_filters, kernel_sizes).to(device)
print(model)


TextCNN(
  (embed): Embedding(299567, 300)
  (convs_1d): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1), padding=(3, 0))
  )
  (fc): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [27]:
import torch.optim as optim
optimizer = optim.Adadelta(model.parameters()) #Adadelta 是一种自适应学习率优化算法，不需要显式设置全局学习率

In [28]:
#开始训练
import time
for epoch in range(1, epochs + 1):
    start_time = time.time()  #记录开始时间
    train([], model, device, train_loader, optimizer, epoch)  #训练
    train_time = time.time()
    test(model, device, valid_loader)  #测试
    test_time = time.time()
    train_duration = train_time - start_time
    test_duration = test_time - train_time
    print(f"Epoch {epoch}/{epochs} - Trained in {train_duration:.2f} seconds  - Test in {test_duration:.2f} seconds.\n")

Evaluation Results:
 - Average Loss: 0.5813
 - Accuracy: 432/572 (75.52%)
Epoch 1/6 - Trained in 0.88 seconds  - Test in 0.06 seconds.

Evaluation Results:
 - Average Loss: 0.5267
 - Accuracy: 447/572 (78.15%)
Epoch 2/6 - Trained in 0.50 seconds  - Test in 0.05 seconds.

Evaluation Results:
 - Average Loss: 0.2978
 - Accuracy: 517/572 (90.38%)
Epoch 3/6 - Trained in 0.50 seconds  - Test in 0.04 seconds.

Evaluation Results:
 - Average Loss: 0.2366
 - Accuracy: 532/572 (93.01%)
Epoch 4/6 - Trained in 0.50 seconds  - Test in 0.04 seconds.

Evaluation Results:
 - Average Loss: 0.2047
 - Accuracy: 536/572 (93.71%)
Epoch 5/6 - Trained in 0.50 seconds  - Test in 0.04 seconds.

Evaluation Results:
 - Average Loss: 0.1918
 - Accuracy: 537/572 (93.88%)
Epoch 6/6 - Trained in 0.50 seconds  - Test in 0.04 seconds.



In [29]:
test(model, device, valid_loader)

Evaluation Results:
 - Average Loss: 0.1918
 - Accuracy: 537/572 (93.88%)
